In [ ]:
import flwr as fl
import matplotlib.pyplot as plt
from dataset import load_client_data
from client import FlowerClient

In [ ]:
DATA_ROOT = r"C:\Users\anujs\OneDrive\Desktop\sample_project\Prac_Data_poisoning\anomaly detection Dataset"

CLIENTS = [
    "Danmini_Doorbell",
    "Ecobee_Thermostat",
    "Philips_B120N10_Baby_Monitor",
    "Provision_PT_737E_Security_Camera",
    "Provision_PT_838_Security_Camera"
]
NUM_CLIENTS = 5
ROUNDS = 10
acc_per_round = []

In [ ]:
def client_fn(cid):
    cid = int(cid)
    train_loader, test_loader, input_dim, num_classes = load_client_data(
        DATA_ROOT, CLIENTS[cid]
    )
    poisoned = cid in [1, 3]
    return FlowerClient(
        train_loader,
        test_loader,
        input_dim,
        num_classes,
        poisoned
    )

In [ ]:
def aggregate_metrics(metrics):
    acc = sum(m["accuracy"] for _,m in metrics)/len(metrics)
    acc_per_round.append(acc)
    return {"accuracy": acc}
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    min_fit_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    evaluate_metrics_aggregation_fn=aggregate_metrics
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=ROUNDS),
    strategy=strategy
)


In [ ]:
plt.plot(range(1, ROUNDS + 1), acc_per_round)
plt.xlabel("Round")
plt.ylabel("Accuracy")
plt.title("FedAvg (poisoned (False) Accuracy vs Round")
plt.show()